# Generative example

This example uses the LangChain framework with Ollama to generate synthetic clinical notes from synthetic structured data.

Running this notebook requires an additional install of [Ollama](https://ollama.ai/) and the particular model used is `llama2:latest` (also named `llama2:7b-chat`) from the [Ollama model library](https://ollama.ai/library/llama2/tags).

Open a terminal and run `ollama pull llama2` to download the model.

## Import libraries

In [ ]:
import json

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

## Load input in batch

In [ ]:
input_path = "../../data/synthea_dataset.json"

with open(input_path) as file:
    data = json.load(file)

batch = []

for i in range(5):
    batch.append({"data": json.dumps(data[i])})

## Load LLM

The model used to run inference can be easily swapped out using LangChain and Ollama!

Simply open a terminal and run `ollama pull <model_name:tag>` to retrieve any model from the Ollama model library and pass `<model_name:tag>` as the new argument when instantiating the LLM.

For example, if you wanted to use Mistral instead of Llama2, you would need to run `ollama pull mistral` in a terminal and set `model="mistral"` below.

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = Ollama(model="llama2", callback_manager=callback_manager)

## Construct prompt

In [ ]:
template = """[INST]
<<SYS>>
You are a medical student answering an exam question about writing clinical notes for patients.
<</SYS>>

Keep in mind that your answer will be asssessed based on incorporating all the provided information and the quality of prose.

1. Use prose to write an example clinical note for this patient's doctor.
2. Use less than three sentences.
3. Do not provide a diagnosis or recommendations.
4. Use the following information:

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm

## Run inference

In [ ]:
outputs = chain.batch(batch)